In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder as le
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import metrics 
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
import cvxpy as cp
from cvxopt import solvers, matrix, spdiag, log

In [0]:
homeB = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeB.csv', index_col="Date & Time")
homeC = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeC.csv', index_col="Date & Time")
homeF = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeF.csv', index_col="Date & Time")

In [0]:
homeB_LR = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeB_LR.csv', index_col="Date & Time")
homeC_LR = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeC_LR.csv', index_col="Date & Time")
homeF_LR = pd.read_csv(r'/content/gdrive/My Drive/Smart Energy/Assign2/homeF_LR.csv', index_col="Date & Time")

**Task 1: Solve the offline optimization problem, e.g., using tools CVX in Matlab or Python.** 

In [0]:
def offline_optimisation_problem(home, type = 'static', p_t = 0.40,  a = 4,  b = 4):
  
  obj = 0
  y = home.Total_consumption[1:673]

  if type == 'static':
    x = cp.Variable(1)
    for i in range(len(y)):
      obj = obj + (p_t*x + a*cp.maximum(0, y.values[i]-x))

  elif type == 'dynamic':
    x = cp.Variable(672)
    for i in range(len(y)):
      obj = obj + (p_t*x[i] + a*cp.maximum(0, y.values[i]-x[i]) + b*cp.abs(x[i] - x[i-1]))
  
  constraints = [0 <= x]
  min_obj = cp.Minimize(obj)
  problem = cp.Problem(min_obj, constraints) 
  result = problem.solve() 
  print("Offline Optimization Problem solution for "+type+" is: ", result)

  return [result, x.value]

In [35]:
print("For home B:")
[offline_static_result_B,offline_static_x_values_B] = offline_optimisation_problem(homeB,'static', p_t = 0.40,  a = 4,  b = 4)
[offline_dynamic_result_B,offline_dynamic_x_values_B] = offline_optimisation_problem(homeB, 'dynamic', p_t = 0.40,  a = 4,  b = 4)
print("For home C:")
[offline_static_result_C,offline_static_x_values_C] = offline_optimisation_problem(homeC,'static', p_t = 0.40,  a = 4,  b = 4)
[offline_dynamic_result_C,offline_dynamic_x_values_C] = offline_optimisation_problem(homeC, 'dynamic', p_t = 0.40,  a = 4,  b = 4)
print("For home F:")
[offline_static_result_F,offline_static_x_values_F] = offline_optimisation_problem(homeF,'static', p_t = 0.40,  a = 4,  b = 4)
[offline_dynamic_result_F,offline_dynamic_x_values_F] = offline_optimisation_problem(homeF, 'dynamic', p_t = 0.40,  a = 4,  b = 4)

For home B:
Offline Optimization Problem solution for static is:  1018.4013103622867
Offline Optimization Problem solution for dynamic is:  836.0104660550957
For home C:
Offline Optimization Problem solution for static is:  1365.6704999799842
Offline Optimization Problem solution for dynamic is:  1137.486313296603
For home F:
Offline Optimization Problem solution for static is:  6286.343226100506
Offline Optimization Problem solution for dynamic is:  5232.1411726006945


**Task 2(i): Online gradient descent (with different step size)**

In [0]:
def OGD_online_op(home, p_t = 0.40,  a = 4,  b = 4, step_size_type = 'A'):
  obj = 0
  # prev_result = float('inf')
  y = home.Total_consumption[1:673]
  x = [0]*(len(y)+1)

  obj = 0
  for t in range(len(y)):
    if y[t] > x[t]:
      if x[t] > x[t-1]:
          k = p_t-a+b
      else:
          k = p_t-a-b
    else:
        if x[t] > x[t-1]:
            k = p_t+b
        else:
            k = p_t-b;
    if step_size_type == 'A':                      #step_size_type = 1/sqrt(t)
      x[t+1]  = x[t] - (1/np.sqrt(t+1))*k
    elif step_size_type == 'B':                    #step_size_type = 1/(t)
      x[t+1]  = x[t] - (1/(t+1))*k
    elif step_size_type == 'C':                    #step_size_type = 0.01
      x[t+1]  = x[t] - (0.01)*k
    elif step_size_type == 'D':                    #step_size_type = 1/672
      x[t+1]  = x[t] - (1/672)*k

    #x[t+1]  = x[t] - neta_derivative(t,x[t],x[t-1])

  for t in range(len(y)):
    obj = obj + (p_t*x[t] + a*cp.maximum(0, y.values[t]-x[t]) + b*cp.abs(x[t] - x[t-1]))
    

  min_obj = cp.Minimize(obj)
  problem = cp.Problem(min_obj) 
  result = problem.solve()
  print("OGD Optimization Problem solution is: ", result)
   
  return result


step_size_type = 'A': 1/sqrt(t)

In [53]:
print("For Home B,")
OGD_results_B = OGD_online_op(homeB, p_t = 0.40, a =  4, b= 4,step_size_type = 'A')
print("For Home C,")
OGD_results_C = OGD_online_op(homeC, p_t = 0.40, a =  4, b= 4,step_size_type = 'A')
print("For Home F,")
OGD_results_F = OGD_online_op(homeF, p_t = 0.40, a =  4, b= 4,step_size_type = 'A')

For Home B,
OGD Optimization Problem solution is:  1892.720581127459
For Home C,
OGD Optimization Problem solution is:  2143.127367233312
For Home F,
OGD Optimization Problem solution is:  7521.536038487382


step_size_type = 'B': 1/(t)

In [164]:
print("For Home B,")
OGD_results_B = OGD_online_op(homeB, p_t = 0.40, a =  4, b= 4,step_size_type = 'B')
print("For Home C,")
OGD_results_C = OGD_online_op(homeC, p_t = 0.40, a =  4, b= 4,step_size_type = 'B')
print("For Home F,")
OGD_results_F = OGD_online_op(homeF, p_t = 0.40, a =  4, b= 4,step_size_type = 'B')

For Home B,
OGD Optimization Problem solution is:  1583.378957866783
For Home C,
OGD Optimization Problem solution is:  1614.6201205975656
For Home F,
OGD Optimization Problem solution is:  10516.319202234305


step_size_type = 'C': 0.01

In [165]:
print("For Home B,")
OGD_results_B = OGD_online_op(homeB, p_t = 0.40, a =  4, b= 4,step_size_type = 'C')
print("For Home C,")
OGD_results_C = OGD_online_op(homeC, p_t = 0.40, a =  4, b= 4,step_size_type = 'C')
print("For Home F,")
OGD_results_F = OGD_online_op(homeF, p_t = 0.40, a =  4, b= 4,step_size_type = 'C')

For Home B,
OGD Optimization Problem solution is:  1244.8327133920006
For Home C,
OGD Optimization Problem solution is:  1771.6468845240013
For Home F,
OGD Optimization Problem solution is:  12635.242333375993


step_size_type = 'D': 1/672

In [166]:
print("For Home B,")
OGD_results_B = OGD_online_op(homeB, p_t = 0.40, a =  4, b= 4,step_size_type = 'D')
print("For Home C,")
OGD_results_C = OGD_online_op(homeC, p_t = 0.40, a =  4, b= 4,step_size_type = 'D')
print("For Home F,")
OGD_results_F = OGD_online_op(homeF, p_t = 0.40, a =  4, b= 4,step_size_type = 'D')

For Home B,
OGD Optimization Problem solution is:  1744.7828514939033
For Home C,
OGD Optimization Problem solution is:  2804.8918305921875
For Home F,
OGD Optimization Problem solution is:  17120.758057329414


## **Comment about different Step size of OGD for different home:**

For Home B, best step size is **step_size_type = 'C': 0.01** with **minimum cost = 1244.8327133920006**

For Home C, best step size is **step_size_type = 'B': 1/(t)** with **minimum cost = 1614.6201205975656**

For Home F, best step size is **step_size_type = 'A': 1/sqrt(t)** with **minimum cost = 7521.536038487382**

**Task 2(ii):  Receding horizon control (with different prediction window size)**

In [0]:
def control_algo_online_op(home, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5,commitmentHorizon = 1):
  
  y = home.y_pred[1:]    #Using Predicted Values
  result1 = []
  optValues = [0]*(len(y))
  length = min(672, len(y))
  for horizonStart in range(length):
    horizonEnd = horizonStart + predictionHorizon
    windowY = y[horizonStart: horizonEnd]
    
    obj = 0
    x = cp.Variable(predictionHorizon)

    for i in range(predictionHorizon):
        obj = obj + p_t*x[i] + a*cp.maximum(0,windowY[i]-x[i]);
        if i == 0:
            obj = obj + b*cp.abs(x[i])
        else:
            obj = obj + b*cp.abs(x[i]-x[i-1])
            
    min_obj = cp.Minimize(obj)
    problem = cp.Problem(min_obj) 
    result1.append(problem.solve())

    for i in range(commitmentHorizon):
      optValues[horizonStart+i] = optValues[horizonStart+i] + x[i]

  optValues = np.dot(optValues,1/commitmentHorizon)

  obj = 0
  for i in range(length):
    obj = obj + p_t*optValues[i] + a*cp.maximum(0,y[i]-optValues[i]);
    if i == 0:
        obj = obj + b*cp.abs(optValues[i])
    else:
        obj = obj + b*cp.abs(optValues[i]-optValues[i-1])

  min_obj = cp.Minimize(obj)
  problem = cp.Problem(min_obj) 
  result2 = problem.solve()

  return [optValues, result2]



For ARIMA Model Predictions:

In [171]:
window_size = [3,5,10]
for window in window_size:
  print("For window_size: ", window)
  [optimal_values_RHC_B, result_RHC_B] = control_algo_online_op(homeB, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home B, RHC Optimization Problem solution is: ", result_RHC_B)
  [optimal_values_RHC_C, result_RHC_C] = control_algo_online_op(homeC, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home C, RHC Optimization Problem solution is: ", result_RHC_C)
  [optimal_values_RHC_F, result_RHC_F] = control_algo_online_op(homeF, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home F, RHC Optimization Problem solution is: ", result_RHC_F)

For window_size:  3
For Home B, RHC Optimization Problem solution is:  750.3907458132975
For Home C, RHC Optimization Problem solution is:  989.8302788430145
For Home F, RHC Optimization Problem solution is:  4551.13984721234
For window_size:  5
For Home B, RHC Optimization Problem solution is:  750.3907458132975
For Home C, RHC Optimization Problem solution is:  989.8302788430145
For Home F, RHC Optimization Problem solution is:  4551.13984721234
For window_size:  10
For Home B, RHC Optimization Problem solution is:  750.3907458132975
For Home C, RHC Optimization Problem solution is:  989.8302788430145
For Home F, RHC Optimization Problem solution is:  4551.13984721234


For Linear Regression Model Predictions:

In [172]:
window_size = [3,5,10]
for window in window_size:
  print("For window_size: ", window)
  [optimal_values_RHC_B_LR, result_RHC_B_LR] = control_algo_online_op(homeB_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home B, RHC Optimization Problem solution is: ", result_RHC_B_LR)
  [optimal_values_RHC_C_LR, result_RHC_C_LR] = control_algo_online_op(homeC_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home C, RHC Optimization Problem solution is: ", result_RHC_C_LR)
  [optimal_values_RHC_F_LR, result_RHC_F_LR] = control_algo_online_op(homeF_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = window)
  print("For Home F, RHC Optimization Problem solution is: ", result_RHC_F_LR)

For window_size:  3
For Home B, RHC Optimization Problem solution is:  721.894394948552
For Home C, RHC Optimization Problem solution is:  1005.4272601940075
For Home F, RHC Optimization Problem solution is:  4414.920796928689
For window_size:  5
For Home B, RHC Optimization Problem solution is:  721.894394948552
For Home C, RHC Optimization Problem solution is:  1005.4272601940075
For Home F, RHC Optimization Problem solution is:  4414.920796928689
For window_size:  10
For Home B, RHC Optimization Problem solution is:  721.894394948552
For Home C, RHC Optimization Problem solution is:  1005.4272601940075
For Home F, RHC Optimization Problem solution is:  4414.920796928689


## **Comment about Best window_size of RHC:**
For ARIMA Predictions:

For window_size:  5

For Home B, RHC Optimization Problem solution is:  750.3907458132975

For Home C, RHC Optimization Problem solution is:  989.8302788430145

For Home F, RHC Optimization Problem solution is:  4551.13984721234

For Linear Regression Predictions:

For window_size:  5

For Home B, RHC Optimization Problem solution is:  721.894394948552

For Home C, RHC Optimization Problem solution is:  1005.4272601940075

For Home F, RHC Optimization Problem solution is:  4414.920796928689

**Task 2(iii):  Commitment Horizon control (with different commitment levels)**

For ARIMA Model Predictions:

In [173]:
commitmentHorizon = [3,4,5]
for commitment in commitmentHorizon:
  print("For v: ", commitment)
  [optimal_values_CHC_B, result_CHC_B] = control_algo_online_op(homeB, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= commitment)
  print("For Home B, CHC Optimization Problem solution is: ", result_CHC_B)
  [optimal_values_CHC_C, result_CHC_C] = control_algo_online_op(homeC, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= commitment)
  print("For Home C, CHC Optimization Problem solution is: ", result_CHC_C)
  [optimal_values_CHC_F, result_CHC_F] = control_algo_online_op(homeF, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= commitment)
  print("For Home F, CHC Optimization Problem solution is: ", result_CHC_F)

For v:  3
For Home B, CHC Optimization Problem solution is:  750.3907458133029
For Home C, CHC Optimization Problem solution is:  989.8302788430127
For Home F, CHC Optimization Problem solution is:  4551.139847212367
For v:  4
For Home B, CHC Optimization Problem solution is:  750.3907458133042
For Home C, CHC Optimization Problem solution is:  989.8302788430152
For Home F, CHC Optimization Problem solution is:  4551.139847212363
For v:  5
For Home B, CHC Optimization Problem solution is:  750.3907458133061
For Home C, CHC Optimization Problem solution is:  989.8302788430143
For Home F, CHC Optimization Problem solution is:  4551.139847212366


For Linear Regression Model Predictions:

In [174]:
commitmentHorizon = [3,4,5]
for commitment in commitmentHorizon:
  print("For v: ", commitment)
  [optimal_values_CHC_B_LR, result_CHC_B_LR] = control_algo_online_op(homeB_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= 3)
  print("For Home B, CHC Optimization Problem solution is: ", result_CHC_B_LR)
  [optimal_values_CHC_C_LR, result_CHC_C_LR] = control_algo_online_op(homeC_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= 3)
  print("For Home C, CHC Optimization Problem solution is: ", result_CHC_C_LR)
  [optimal_values_CHC_F_LR, result_CHC_F_LR] = control_algo_online_op(homeF_LR, p_t = 0.40, a = 4, b = 4, predictionHorizon = 5, commitmentHorizon= 3)
  print("For Home F, CHC Optimization Problem solution is: ", result_CHC_F_LR)

For v:  3
For Home B, CHC Optimization Problem solution is:  721.8943949485557
For Home C, CHC Optimization Problem solution is:  1005.4272601940361
For Home F, CHC Optimization Problem solution is:  4414.920796928767
For v:  4
For Home B, CHC Optimization Problem solution is:  721.8943949485557
For Home C, CHC Optimization Problem solution is:  1005.4272601940361
For Home F, CHC Optimization Problem solution is:  4414.920796928767
For v:  5
For Home B, CHC Optimization Problem solution is:  721.8943949485557
For Home C, CHC Optimization Problem solution is:  1005.4272601940361
For Home F, CHC Optimization Problem solution is:  4414.920796928767


## **Comment about Best Commitment Horizon for CHC:**


**For ARIMA Predictions:**

For v:  3

For Home B, CHC Optimization Problem solution is:  750.3907458133029

For Home C, CHC Optimization Problem solution is:  989.8302788430127

For Home F, CHC Optimization Problem solution is:  4551.139847212367

**For Linear Regression Predictions:**

For v:  3

For Home B, CHC Optimization Problem solution is:  721.8943949485557

For Home C, CHC Optimization Problem solution is:  1005.4272601940361

For Home F, CHC Optimization Problem solution is:  4414.920796928767

**Task 3: Compare the costs of these algorithms to those of the offline static and dynamic solutions: Calculating Regret**

**Regrets for OGD**

In [177]:
print("For Home B,")
OGD_results_B = OGD_online_op(homeB, p_t = 0.40, a =  4, b= 4,step_size_type = 'C')
print("For Home C,")
OGD_results_C = OGD_online_op(homeC, p_t = 0.40, a =  4, b= 4,step_size_type = 'B')
print("For Home F,")
OGD_results_F = OGD_online_op(homeF, p_t = 0.40, a =  4, b= 4,step_size_type = 'A')

For Home B,
OGD Optimization Problem solution is:  1244.8327133920006
For Home C,
OGD Optimization Problem solution is:  1614.6201205975656
For Home F,
OGD Optimization Problem solution is:  7521.536038487382


In [178]:
static_regret_OGD_B = OGD_results_B - offline_static_result_B
static_regret_OGD_C = OGD_results_C - offline_static_result_C
static_regret_OGD_F = OGD_results_F - offline_static_result_F
print("For Home B, static Regret: ", static_regret_OGD_B)
print("For Home C, static Regret: ", static_regret_OGD_C)
print("For Home F, static Regret: ", static_regret_OGD_F)
dynamic_regret_OGD_B = OGD_results_B - offline_dynamic_result_B
dynamic_regret_OGD_C = OGD_results_C - offline_dynamic_result_C
dynamic_regret_OGD_F = OGD_results_F - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_OGD_B)
print("For Home C, dynamic Regret: ", dynamic_regret_OGD_C)
print("For Home F, dynamic Regret: ", dynamic_regret_OGD_F)

For Home B, static Regret:  226.43140302971392
For Home C, static Regret:  248.94962061758133
For Home F, static Regret:  1235.1928123868756
For Home B, dynamic Regret:  408.822247336905
For Home C, dynamic Regret:  477.13380730096264
For Home F, dynamic Regret:  2289.3948658866875


**Regrets for RHC for ARIMA:**

In [179]:
static_regret_RHC_B = result_RHC_B - offline_static_result_B
static_regret_RHC_C = result_RHC_C - offline_static_result_C
static_regret_RHC_F = result_RHC_F - offline_static_result_F
print("For Home B, static Regret: ", static_regret_RHC_B)
print("For Home C, static Regret: ", static_regret_RHC_C)
print("For Home F, static Regret: ", static_regret_RHC_F)
dynamic_regret_RHC_B = result_RHC_B - offline_dynamic_result_B
dynamic_regret_RHC_C = result_RHC_C - offline_dynamic_result_C
dynamic_regret_RHC_F = result_RHC_F - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_RHC_B)
print("For Home C, dynamic Regret: ", dynamic_regret_RHC_C)
print("For Home F, dynamic Regret: ", dynamic_regret_RHC_F)

For Home B, static Regret:  -268.01056454898924
For Home C, static Regret:  -375.8402211369697
For Home F, static Regret:  -1735.2033788881663
For Home B, dynamic Regret:  -85.61972024179818
For Home C, dynamic Regret:  -147.6560344535884
For Home F, dynamic Regret:  -681.0013253883544


**Regrets for RHC for Linear Regression:**

In [180]:
static_regret_RHC_B_LR = result_RHC_B_LR - offline_static_result_B
static_regret_RHC_C_LR = result_RHC_C_LR - offline_static_result_C
static_regret_RHC_F_LR = result_RHC_F_LR - offline_static_result_F
print("For Home B, static Regret: ", static_regret_RHC_B_LR)
print("For Home C, static Regret: ", static_regret_RHC_C_LR)
print("For Home F, static Regret: ", static_regret_RHC_F_LR)
dynamic_regret_RHC_B_LR = result_RHC_B_LR - offline_dynamic_result_B
dynamic_regret_RHC_C_LR = result_RHC_C_LR - offline_dynamic_result_C
dynamic_regret_RHC_F_LR = result_RHC_F_LR - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_RHC_B_LR)
print("For Home C, dynamic Regret: ", dynamic_regret_RHC_C_LR)
print("For Home F, dynamic Regret: ", dynamic_regret_RHC_F_LR)

For Home B, static Regret:  -296.50691541373476
For Home C, static Regret:  -360.2432397859767
For Home F, static Regret:  -1871.4224291718174
For Home B, dynamic Regret:  -114.1160711065437
For Home C, dynamic Regret:  -132.05905310259539
For Home F, dynamic Regret:  -817.2203756720055


**Regrets for CHC for ARIMA:**

In [181]:
static_regret_CHC_B = result_CHC_B - offline_static_result_B
static_regret_CHC_C = result_CHC_C - offline_static_result_C
static_regret_CHC_F = result_CHC_F - offline_static_result_F
print("For Home B, static Regret: ", static_regret_CHC_B)
print("For Home C, static Regret: ", static_regret_CHC_C)
print("For Home F, static Regret: ", static_regret_CHC_F)
dynamic_regret_CHC_B = result_CHC_B - offline_dynamic_result_B
dynamic_regret_CHC_C = result_CHC_C - offline_dynamic_result_C
dynamic_regret_CHC_F = result_CHC_F - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_CHC_B)
print("For Home C, dynamic Regret: ", dynamic_regret_CHC_C)
print("For Home F, dynamic Regret: ", dynamic_regret_CHC_F)

For Home B, static Regret:  -268.0105645489806
For Home C, static Regret:  -375.84022113696994
For Home F, static Regret:  -1735.2033788881408
For Home B, dynamic Regret:  -85.61972024178954
For Home C, dynamic Regret:  -147.65603445358863
For Home F, dynamic Regret:  -681.001325388329


**Regrets for CHC for Linear Regression:**

In [182]:
static_regret_CHC_B_LR = result_CHC_B_LR - offline_static_result_B
static_regret_CHC_C_LR = result_CHC_C_LR - offline_static_result_C
static_regret_CHC_F_LR = result_CHC_F_LR - offline_static_result_F
print("For Home B, static Regret: ", static_regret_CHC_B_LR)
print("For Home C, static Regret: ", static_regret_CHC_C_LR)
print("For Home F, static Regret: ", static_regret_CHC_F_LR)
dynamic_regret_CHC_B_LR = result_CHC_B_LR - offline_dynamic_result_B
dynamic_regret_CHC_C_LR = result_CHC_C_LR - offline_dynamic_result_C
dynamic_regret_CHC_F_LR = result_CHC_F_LR - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_CHC_B_LR)
print("For Home C, dynamic Regret: ", dynamic_regret_CHC_C_LR)
print("For Home F, dynamic Regret: ", dynamic_regret_CHC_F_LR)

For Home B, static Regret:  -296.506915413731
For Home C, static Regret:  -360.24323978594816
For Home F, static Regret:  -1871.4224291717392
For Home B, dynamic Regret:  -114.11607110653995
For Home C, dynamic Regret:  -132.05905310256685
For Home F, dynamic Regret:  -817.2203756719273


**Task 4: For the best combination of control algorithm and prediction algorithm, vary a and b to see the
impacts**

In [0]:
A = [4, 0.1, 1, 5, 10]
B = [4, 0.1, 1, 5, 10]

In [188]:
for ai in A:
  for bi in B:
    print("For a = ",ai,"and b = ",bi,":")
    [_, result_B] = control_algo_online_op(homeB_LR, p_t = 0.40,  a = ai,  b = bi, predictionHorizon = 5, commitmentHorizon=3)  #Linear Regression predictions with CHC
    print("For Home B, Result of CHC with LR(w=5,v=3): ", result_B)
    [_, result_C] = control_algo_online_op(homeC, p_t = 0.40, a = ai,  b = bi, predictionHorizon = 5, commitmentHorizon=3)     #ARIMA predictions with CHC
    print("For Home C, Result of CHC with ARIMA(w=5,v=3): ", result_C)
    [_, result_F] = control_algo_online_op(homeF_LR, p_t = 0.40, a = ai,  b = bi, predictionHorizon = 5, commitmentHorizon=3)  #Linear Regression predictions with CHC
    print("For Home F, Result of CHC with LR(w=5,v=3): ", result_F)

For a =  4 and b =  4 :
For Home B, Result of CHC with LR(w=5,v=3):  721.8943949485557
For Home C, Result of CHC with ARIMA(w=5,v=3):  989.8302788430127
For Home F, Result of CHC with LR(w=5,v=3):  4414.920796928767
For a =  4 and b =  0.1 :
For Home B, Result of CHC with LR(w=5,v=3):  420.8331773104108
For Home C, Result of CHC with ARIMA(w=5,v=3):  609.9011831912904
For Home F, Result of CHC with LR(w=5,v=3):  2238.5675645420356
For a =  4 and b =  1 :
For Home B, Result of CHC with LR(w=5,v=3):  545.7380264463891
For Home C, Result of CHC with ARIMA(w=5,v=3):  780.3340307005786
For Home F, Result of CHC with LR(w=5,v=3):  3033.576477375311
For a =  4 and b =  5 :
For Home B, Result of CHC with LR(w=5,v=3):  747.7932221815256
For Home C, Result of CHC with ARIMA(w=5,v=3):  1015.2364635092357
For Home F, Result of CHC with LR(w=5,v=3):  4682.126267785505
For a =  4 and b =  10 :
For Home B, Result of CHC with LR(w=5,v=3):  837.638343951412
For Home C, Result of CHC with ARIMA(w=5,v=3)

## **Best parameters**

•	Best for home B:

    For a = 4 and b = 0.1:

    For Home B, Result of CHC with Linear Regression (w=5, v=3):  420.8331773104108

•	Best for home C:

    For a = 4 and b = 0.1:

    For Home C, Result of CHC with ARIMA(w=5,v=3):  609.9011831912904

•	Best for home F:

    For a = 4 and b = 0.1:

    For Home F, Result of CHC with LR(w=5,v=3):  2238.5675645420356

**Task 5: Try at least two algorithm selection (one deterministic, one randomized) to see if their
performance**

**Deterministic: Weighted Majority**

In [0]:
w = [0.33, 0.33, 0.33]
regret = []
test_home = homeB[::-1]

In [141]:
for i in range(4):
  regret = []
  if i == 0:      
    X1 = OGD_online_op(homeB) #OGD
    _, X2 = control_algo_online_op(homeB) #RHC
    _ , X3 = control_algo_online_op(homeB, commitmentHorizon= 4)  #CHC
    test_offline_static,_ = offline_optimisation_problem(homeB, type='static',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_static)
    regret.append(X2-test_offline_static)
    regret.append(X3-test_offline_static)
    w[regret.index(min(regret))] = w[regret.index(min(regret))]+0.10
    w[regret.index(max(regret))] = w[regret.index(min(regret))]-0.10
  elif i == 1:
    test_offline_dynamic,_ = offline_optimisation_problem(homeB, type='dynamic',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_dynamic)
    regret.append(X2-test_offline_dynamic)
    regret.append(X3-test_offline_dynamic)
    w[regret.index(min(regret))] = w[regret.index(min(regret))]+0.10
    w[regret.index(max(regret))] = w[regret.index(min(regret))]-0.10
  elif i == 2:
    X1 = OGD_online_op(test_home) #OGD
    _, X2 = control_algo_online_op(test_home) #RHC
    _ , X3 = control_algo_online_op(test_home, commitmentHorizon= 4)
    test_offline_static,_ = offline_optimisation_problem(test_home, type='static',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_static)
    regret.append(X2-test_offline_static)
    regret.append(X3-test_offline_static)
    w[regret.index(min(regret))] = w[regret.index(min(regret))]+0.10
    w[regret.index(max(regret))] = w[regret.index(min(regret))]-0.10
  else:
    test_offline_dynamic,_ = offline_optimisation_problem(test_home, type='dynamic',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_dynamic)
    regret.append(X2-test_offline_dynamic)
    regret.append(X3-test_offline_dynamic)
    w[regret.index(min(regret))] = w[regret.index(min(regret))]+0.10
    w[regret.index(max(regret))] = w[regret.index(min(regret))]-0.10
    

OGD Optimization Problem solution is:  1892.720581127459
Offline Optimization Problem solution for static is:  1018.4013103622867
Offline Optimization Problem solution for dynamic is:  836.0104660550957
OGD Optimization Problem solution is:  1837.2775949715374
Offline Optimization Problem solution for static is:  949.3810790959524
Offline Optimization Problem solution for dynamic is:  776.0704729923808


In [142]:
print("Algorithm Selection Using Weighted Majority:")
print(str(w[0])+"*OGD + "+str(w[1])+"*RHC + "+str(w[2])+"*CHC")
print("We choose RHC as it has maximum weight: ", w[1])

Algorithm Selection Using Weighted Majority:
0.63*OGD + 0.73*RHC + 0.33*CHC
We choose RHC as it has maximum weight:  0.73


**Randomized Algorithm Selection**

In [0]:
test_home = homeB[::-1]

In [160]:
winning = [0,0,0]
for i in range(4):
  regret = []
  random_num = np.random.randint(4)
  print(random_num)
  if random_num == 0:      
    X1 = OGD_online_op(homeB) #OGD
    _, X2 = control_algo_online_op(homeB) #RHC
    _ , X3 = control_algo_online_op(homeB, commitmentHorizon= 4)  #CHC
    test_offline_static,_ = offline_optimisation_problem(homeB, type='static',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_static)
    regret.append(X2-test_offline_static)
    regret.append(X3-test_offline_static)
    winning[regret.index(min(regret))] = winning[regret.index(min(regret))]+1
  elif random_num == 1:
    X1 = OGD_online_op(homeB) #OGD
    _, X2 = control_algo_online_op(homeB) #RHC
    _ , X3 = control_algo_online_op(homeB, commitmentHorizon= 4)  #CHC
    test_offline_dynamic,_ = offline_optimisation_problem(homeB, type='dynamic',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_dynamic)
    regret.append(X2-test_offline_dynamic)
    regret.append(X3-test_offline_dynamic)
    winning[regret.index(min(regret))] = winning[regret.index(min(regret))]+1
  elif random_num == 2:
    X1 = OGD_online_op(test_home) #OGD
    _, X2 = control_algo_online_op(test_home) #RHC
    _ , X3 = control_algo_online_op(test_home, commitmentHorizon= 4)
    test_offline_static,_ = offline_optimisation_problem(test_home, type='static',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_static)
    regret.append(X2-test_offline_static)
    regret.append(X3-test_offline_static)
    winning[regret.index(min(regret))] = winning[regret.index(min(regret))]+1
  else:
    X1 = OGD_online_op(test_home) #OGD
    _, X2 = control_algo_online_op(test_home) #RHC
    _ , X3 = control_algo_online_op(test_home, commitmentHorizon= 4)
    test_offline_dynamic,_ = offline_optimisation_problem(test_home, type='dynamic',p_t = 0.4, a = 4, b=4)
    regret.append(X1-test_offline_dynamic)
    regret.append(X2-test_offline_dynamic)
    regret.append(X3-test_offline_dynamic)
    winning[regret.index(min(regret))] = winning[regret.index(min(regret))]+1

0
OGD Optimization Problem solution is:  1892.720581127459
Offline Optimization Problem solution for static is:  1018.4013103622867
2
OGD Optimization Problem solution is:  1837.2775949715374
Offline Optimization Problem solution for static is:  949.3810790959524
2
OGD Optimization Problem solution is:  1837.2775949715374
Offline Optimization Problem solution for static is:  949.3810790959524
3
OGD Optimization Problem solution is:  1837.2775949715374
Offline Optimization Problem solution for dynamic is:  776.0704729923808


In [161]:
print("Algorithm Selection Using Randomized Maximum Winning")
print("Winning of OGD", winning[0])
print("Winning of RHC", winning[1])
print("Winning of CHC", winning[2])

Algorithm Selection Using Randomized Maximum Winning
Winning of OGD 0
Winning of RHC 4
Winning of CHC 0


# **Bonus**

## **Online Balanced Descent Algorithm:**


OBD is one step ahead of OGD as its step sizes varies with time.

Reference: https://arxiv.org/pdf/1803.10366.pdf

We have already implemented OBD above.



In [0]:
def OBD_online_op(home, p_t = 0.40,  a = 4,  b = 4, step_size_type = 'A'):
  obj = 0
  # prev_result = float('inf')
  y = home.Total_consumption[1:673]
  x = [0]*(len(y)+1)

  obj = 0
  for t in range(len(y)):
    if y[t] > x[t]:
      if x[t] > x[t-1]:
          k = p_t-a+b
      else:
          k = p_t-a-b
    else:
        if x[t] > x[t-1]:
            k = p_t+b
        else:
            k = p_t-b;
    if step_size_type == 'A':                      #step_size_type = 1/sqrt(t)    Changing with time t
      x[t+1]  = x[t] - (1/np.sqrt(t+1))*k
    elif step_size_type == 'B':                    #step_size_type = 1/(t)  Changing with time t
      x[t+1]  = x[t] - (1/(t+1))*k

    #x[t+1]  = x[t] - neta_derivative(t,x[t],x[t-1])

  for t in range(len(y)):
    obj = obj + (p_t*x[t] + a*cp.maximum(0, y.values[t]-x[t]) + b*cp.abs(x[t] - x[t-1]))
    

  min_obj = cp.Minimize(obj)
  problem = cp.Problem(min_obj) 
  result = problem.solve()
  print("OGD Optimization Problem solution is: ", result)
   
  return result


In [192]:
result_OBD_B = OBD_online_op(homeB)
result_OBD_C = OBD_online_op(homeC)
result_OBD_F = OBD_online_op(homeF)

OGD Optimization Problem solution is:  1892.720581127459
OGD Optimization Problem solution is:  2143.127367233312
OGD Optimization Problem solution is:  7521.536038487382


### **Regrets of OBD:**

In [193]:
static_regret_OGD_B = result_OBD_B - offline_static_result_B
static_regret_OGD_C = result_OBD_C - offline_static_result_C
static_regret_OGD_F = result_OBD_F - offline_static_result_F
print("For Home B, static Regret: ", static_regret_OGD_B)
print("For Home C, static Regret: ", static_regret_OGD_C)
print("For Home F, static Regret: ", static_regret_OGD_F)
dynamic_regret_OGD_B = result_OBD_B - offline_dynamic_result_B
dynamic_regret_OGD_C = result_OBD_C - offline_dynamic_result_C
dynamic_regret_OGD_F = result_OBD_F - offline_dynamic_result_F
print("For Home B, dynamic Regret: ", dynamic_regret_OGD_B)
print("For Home C, dynamic Regret: ", dynamic_regret_OGD_C)
print("For Home F, dynamic Regret: ", dynamic_regret_OGD_F)

For Home B, static Regret:  874.3192707651722
For Home C, static Regret:  777.4568672533276
For Home F, static Regret:  1235.1928123868756
For Home B, dynamic Regret:  1056.7101150723634
For Home C, dynamic Regret:  1005.6410539367089
For Home F, dynamic Regret:  2289.3948658866875


## **Another Algorithm selection Method**

Based on Regret Factor as scale, we can generate new algorithm Selector. In this **Randomized method**, we allocate few keys to all the three algorithms and then we generate a random number if that matches with the key of the algorithm then we going to increase the wins for that algorithm.

In [0]:
#OGD = 0, #OBD = 1 #RHC = 2 #CHC  = 3 
rand_dict = {'OGD': 0, 'OBD': 0, 'RHC':0,'CHC' : 0 }

In [204]:
i = 1
while(i<1000):
  random_number = np.random.randint(4)
  if random_number == 0:
    rand_dict['OGD'] += 1
  elif random_number == 1:
    rand_dict['OBD'] += 1
  elif random_number == 2:
    rand_dict['RHC'] += 1
  else:
    rand_dict['CHC'] += 1
  i += 1
print(rand_dict)

{'OGD': 287, 'OBD': 296, 'RHC': 330, 'CHC': 288}


So, after 1000 trials, RHC got maximum wins. Thus, it will be chosen randomly.